# 🧪 LAB — Concurrencia y Multiproceso

En este laboratorio practicarás los conceptos clave:

- Threads y condiciones de carrera
- Locks
- Multiprocessing y paralelismo real
- Memoria compartida
- Comunicación entre procesos
- Pool de procesos

Cada ejercicio construye una parte de un *pipeline* concurrente.

---

# 1. Condición de carrera sin Lock

Demuestra que varios threads modificando una variable global producen resultados incorrectos.

In [ ]:
import threading

contador = 0

def incrementar():
    global contador
    for _ in range(100000):
        contador += 1

h1 = threading.Thread(target=incrementar)
h2 = threading.Thread(target=incrementar)

h1.start(); h2.start()
h1.join(); h2.join()

contador

# 2. Corrección usando un Lock


In [ ]:
contador = 0
lock = threading.Lock()

def incrementar_lock():
    global contador
    for _ in range(100000):
        with lock:
            contador += 1

h1 = threading.Thread(target=incrementar_lock)
h2 = threading.Thread(target=incrementar_lock)
h1.start(); h2.start()
h1.join(); h2.join()

contador

# 3. Escritura concurrente en fichero

Cada thread debe escribir su identificador en un archivo.

Objetivo: evitar que las líneas se mezclen usando un Lock.

In [ ]:
lock = threading.Lock()

with open('log_threads.txt', 'w') as f:
    pass

def escribir(i):
    with lock:
        with open('log_threads.txt', 'a') as f:
            f.write(f'Thread {i} escribiendo\n')

hilos = []
for i in range(10):
    h = threading.Thread(target=escribir, args=(i,))
    h.start(); hilos.append(h)

for h in hilos: h.join()

open('log_threads.txt').read().splitlines()[:10]

# 4. Multiprocessing: calcular primos en paralelo

Ejecuta una tarea CPU-bound intensa utilizando `Pool`.


In [ ]:
from multiprocessing import Pool
import math

def es_primo(n):
    if n < 2:
        return False
    for i in range(2, int(math.sqrt(n)) + 1):
        if n % i == 0:
            return False
    return True

numeros = list(range(10000, 10300))

with Pool() as pool:
    resultado = pool.map(es_primo, numeros)

sum(resultado)

# 5. Memoria compartida con Array


In [ ]:
from multiprocessing import Process, Array

arr = Array('i', range(10))

def modificar(i, arr):
    arr[i] = arr[i] * 2

procesos = []
for i in range(10):
    p = Process(target=modificar, args=(i, arr))
    p.start(); procesos.append(p)

for p in procesos: p.join()

list(arr)

# 6. Queue: comunicacion entre procesos

Cada proceso enviara un mensaje a la queue.

In [ ]:
from multiprocessing import Queue

q = Queue()

def worker(i, q):
    q.put(f'Proceso {i} listo')

procesos = []
for i in range(5):
    p = Process(target=worker, args=(i, q))
    p.start(); procesos.append(p)

for p in procesos: p.join()

mensajes = []
while not q.empty():
    mensajes.append(q.get())

mensajes

# 7. Ejercicio final integrado

Construir un sistema concurrente que:
1. Lanza 5 threads que generan numeros aleatorios y los escriben en una lista compartida (con Lock)
2. Lanza 4 procesos que reciben esa lista y calculan factoriales
3. Devuelven los factoriales por Queue
4. Se imprime el resultado final en orden

Implementalo abajo:

In [ ]:
# TU CODIGO AQUI


# Solucion (oculta)

<details>
<summary>Mostrar solucion</summary>

```python
import random
import threading
from multiprocessing import Process, Queue

lock = threading.Lock()
numeros = []

def producir():
    n = random.randint(5, 12)
    with lock:
        numeros.append(n)

threads = []
for _ in range(5):
    t = threading.Thread(target=producir)
    t.start(); threads.append(t)

for t in threads: t.join()

def factorial(n):
    r = 1
    for i in range(1, n+1):
        r *= i
    return r

def consumir(n, q):
    q.put((n, factorial(n)))

q = Queue()
procs = []
for n in numeros:
    p = Process(target=consumir, args=(n, q))
    p.start(); procs.append(p)

for p in procs: p.join()

resultados = []
while not q.empty():
    resultados.append(q.get())

sorted(resultados)
```

</details>